In [1]:
import torch
import torch.nn as nn # NN networks (CNN, RNN, losses)
import torch.optim as optim # Aptimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F # Activarions func (ReLU, Sigmoid) also included in nn
from torch.utils.data import DataLoader # Dataset manager
import torchvision.datasets as datasets # Datasets
import torchvision.transforms as transforms # Transformation to datasets
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparametrs
epoches = 1
in_channels = 1
num_classes = 10


In [3]:
from turtle import forward


class CNN(nn.Module):
    def __init__(self, in_channesls=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.reshape(x.shape[0], -1)
        
        x = self.fc1(x)
        return x

In [4]:
# Load data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)


In [5]:
# Train model
# tensorboard --logdir src/guides/runs

# Find hyperparametres and plot (tensorboard)
batch_sizes = [128]
learning_rates = [0.001]
classes = [str(i) for i in range(10)]
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        
        model = CNN().to(device=device)
        model.train()
        
        criterion = nn.CrossEntropyLoss()
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        writer = SummaryWriter(f"runs/MNIST/MiniBatcg {batch_size}, lr {learning_rate}")
        for epoch in range(epoches):
            losses = []
            accuracies = []
            for batch_ind, (data, targets) in enumerate(train_loader):
                # Data on cude
                data = data.to(device=device) # ([64, 1, 28, 28])
                targets = targets.to(device=device)
                
                # Forawrd
                scores = model(data) # Equal to model.forward(data)
                loss = criterion(scores, targets)

                # Backprop
                optimizer.zero_grad()
                loss.backward()

                # Gradient descent or adam step
                optimizer.step()
                
                # Clac running train accuracy
                
                features = data.reshape(data.shape[0], -1)
                _, prediction = scores.max(1)
                num_correct = (prediction == targets).sum()
                running_train_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_train_acc)
                losses.append(loss)
                
                # Images
                img_grid = torchvision.utils.make_grid(data)
                
                # Plot to tensorboard
                writer.add_image("mnist_images", img_grid, global_step=step)

                # See how weight change in entire layer
                writer.add_histogram("fc1", model.fc1.weight, global_step=step)

                # Graph the accuracy and loss
                writer.add_scalar("Training Loss", loss, global_step=step)
                writer.add_scalar("training Accuracy", running_train_acc, global_step=step)
                
                # Visualise features
                if batch_ind == 230:
                    class_labels = [classes[label] for label in prediction]
                    writer.add_embedding(features, metadata=class_labels,
                                     label_img=data,# No necessary.unsqueeze(1), # [64, 1, 28, 28] -> [64, 28, 28]
                                     global_step=batch_ind)
                
                step += 1
            writer.add_hparams({"lr": learning_rate, "bsize": batch_size},
                                {"accuracy": sum(accuracies) / len(accuracies),
                                "loss": sum(losses) / len(losses)})
        print(batch_size, learning_rate)


128 0.001
